In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import h5py
import os

from scipy import signal
from scipy import io

import torchaudio

In [ ]:
from models.mae_ast import MAE_AST
%load_ext autoreload
%autoreload 2

In [ ]:
STATE = "chunk_patch_75_12LaterEncoder.pt"
checkpoint = torch.load(STATE)

cfg = checkpoint["cfg"]["model"]
task_cfg = checkpoint["cfg"]["task"]
        
model = MAE_AST(SimpleNamespace(**checkpoint["cfg"]["model"]), SimpleNamespace(**checkpoint["cfg"]["task"]))
model.load_state_dict(checkpoint["model"], strict=True)

In [ ]:
def wav_to_spectrogram(self, wav):
    return torchaudio.compliance.kaldi.fbank(  # Frame shift and length are standard at  10, 25
        waveform=wav,
        sample_frequency=self.sample_rate,
        use_energy=False,
        num_mel_bins=self.feature_dim
    )

In [ ]:
def read_all_seqs(dir='../data', files=None):
    if files is None:
        files = map(lambda filename: dir + '/' + filename, os.listdir(dir))

    seqs = []
    for filename in files:
        with h5py.File(filename, 'r') as f:
            seqs.extend([f[h5py.h5r.get_name(elem, f.id)][:] for elem in f['chirp_sequence_array']])

    return seqs

In [ ]:
seqs = read_all_seqs()
X = seqs[200].T[:, 0:4]
plt.figure(figsize=(16, 4))
plt.title("Mic data")
plt.plot(X)
plt.legend(["Mic 1", "Mic 2", "Mic 3", "Mic 4"])

In [ ]:
## Setup pytorch

GPU_NUM = '0'

device = torch.device(('cuda:' + GPU_NUM) if (torch.cuda.is_available() and GPU_NUM != '-1') else 'cpu')
print("Using device: {}".format(device))
if device == 'cuda':
    print("Device index: {}".format(torch.cuda.current_device()))


In [ ]:
x = X[:, 1]
model = model.to(device)
x = x.to(device)

In [ ]:
features = wav_to_spectrogram(wav.unsqueeze(0))